# Replication of BCI 3 competition winners

[BCI Competition III: Dataset II- Ensemble of
SVMs for BCI P300 Speller (Alain Rakotomamonjy and Vincent Guigue)](https://ieeexplore.ieee.org/document/4454051)

Method description [here](http://www.bbci.de/competition/iii/results/albany/AlainRakotomamonjy_desc.txt)

In [1]:
%matplotlib inline

from scipy.io import loadmat
from scipy import stats, signal

import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import numpy as np
from tqdm import tqdm

In [13]:
char_matrix = [ ['A', 'B', 'C', 'D', 'E', 'F'],
                ['G', 'H', 'I', 'J', 'K', 'L'],
                ['M', 'N', 'O', 'P', 'Q', 'R'],
                ['S', 'T', 'U', 'V', 'W', 'X'],
                ['Y', 'Z', '1', '2', '3', '4'],
                ['5', '6', '7', '8', '9', '_'] ]

def query_row_column(char):
    for i in range(len(char_matrix)):
        for j in range(len(char_matrix[i])):
            if char_matrix[i][j] == char:
                return i + 7, j + 1


def query_char(row, col):
    '''
    Col from 1 to 6
    Row from 7 to 12
    '''
    return char_matrix[row - 7][col - 1]

## Data loading

In [2]:
compressed_raw_train_a = np.load('datasets/train_A.npz', allow_pickle = True)
raw_a_train_data = compressed_raw_train_a.f.arr_0.reshape([1,1])[0,0]

compressed_raw_test_a = np.load('datasets/test_A.npz', allow_pickle = True)
raw_a_test_data = compressed_raw_test_a.f.arr_0.reshape([1,1])[0,0]

In [3]:
def extract_train_data(raw):
    train_data = {}
    train_data['signal'] = raw['Signal']
    train_data['flashing'] = raw['Flashing']
    train_data['target'] = np.array([i for i in raw['TargetChar'][0]])
    train_data['stimulus_code'] = raw['StimulusCode']
    train_data['stimulus_type'] = raw['StimulusType']
    return train_data


def extract_test_data(raw):
    test_data = {}
    test_data['signal'] = raw['Signal']
    test_data['flashing'] = raw['Flashing']
    test_data['target'] = np.array([
        i for i in
        'WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU'
    ])
    test_data['stimulus_code'] = raw['StimulusCode']

    return test_data

In [4]:
train_data_a = extract_train_data(raw_a_train_data)
test_data_a = extract_test_data(raw_a_test_data)

## Data Preprocessing

### 0 - 667 ms after intensification extraction

#### Train Data

In [5]:
freq = 240 # Dataset was digitized at 240 Hz
samples_per_intensification = int(freq * 0.667)

s = train_data_a['signal'].shape
x_train = np.zeros((s[0], 12 * 15, samples_per_intensification, s[2]))
y_train = np.zeros((s[0], 12 * 15))
stimulus_code_train = np.zeros((s[0], 12 * 15), dtype=np.int)
skip = False
count = 0

for s in range(train_data_a['flashing'].shape[1]): # For each sample
  if not skip and train_data_a['flashing'][0, s] == 1:
    skip = True
    x_train[:, count, :, :] = train_data_a['signal'][:, s:s + samples_per_intensification, :]
    y_train[:, count] = train_data_a['stimulus_type'][:, s]
    stimulus_code_train[:, count] = train_data_a['stimulus_code'][:, s]
    count += 1
  elif skip and train_data_a['flashing'][0, s] == 0:
    skip = False

s = x_train.shape
x_train = np.reshape(x_train, (s[0] * s[1], s[2], s[3]))
y_train = np.reshape(y_train, (s[0] * s[1]))
y_train[y_train == 0] = -1

#### Test Data

In [6]:
s = test_data_a['signal'].shape
x_test = np.zeros((s[0], 12 * 15, samples_per_intensification, s[2]))
y_test = np.zeros((s[0], 12 * 15))
stimulus_code_test = np.zeros((s[0], 12 * 15), dtype=np.int)
skip = False
count = 0

for s in range(test_data_a['flashing'].shape[1]): # For each sample
  if not skip and test_data_a['flashing'][0, s] == 1:
    skip = True
    x_test[:, count, :, :] = test_data_a['signal'][:, s:s + samples_per_intensification, :]
    # y_test[:, count] = test_data_a['stimulus_type'][:, s]
    stimulus_code_test[:, count] = test_data_a['stimulus_code'][:, s]
    count += 1
  elif skip and test_data_a['flashing'][0, s] == 0:
    skip = False

s = x_test.shape
x_test = np.reshape(x_test, (s[0] * s[1], s[2], s[3]))
# y_test = np.reshape(y_test, (s[0] * s[1]))
# y_test[y_test == 0] = -1

### 0.1 - 10 Hz 8-order bandpass Chebyshev filter + decimation at 10 Hz



#### Train Data

In [7]:
n_x_train = np.zeros((x_train.shape[0], 14 * x_train.shape[2]))

for c in tqdm(range(x_train.shape[2])):
  n_x_train[:, c * 14 : (c + 1) * 14] = signal.decimate(x_train[:, : , c], 12)

x_train = n_x_train
del n_x_train

100%|██████████| 64/64 [00:07<00:00,  9.03it/s]


#### Test Data

In [8]:
n_x_test = np.zeros((x_test.shape[0], 14 * x_test.shape[2]))

for c in tqdm(range(x_test.shape[2])):
  n_x_test[:, c * 14 : (c + 1) * 14] = signal.decimate(x_test[:, : , c], 12)

x_test = n_x_test
del n_x_test

100%|██████████| 64/64 [00:08<00:00,  7.72it/s]


### Train Partitioning

In [9]:
partitioned_train = np.zeros((x_train.shape[0] // (5 * 12 * 15), 900, x_train.shape[1]))
partitioned_train_target = np.zeros((y_train.shape[0] // (5 * 12 * 15), 900))
for i in range(partitioned_train.shape[0]):
  partitioned_train[i, :, :] = x_train[i * partitioned_train.shape[1] : (i + 1) * partitioned_train.shape[1], :]
  partitioned_train_target[i, :] = y_train[i * partitioned_train_target.shape[1] : (i + 1) * partitioned_train_target.shape[1]]

## Classification

### Ensemble of classifiers

In [10]:
svmArray = []

for i in tqdm(range(partitioned_train.shape[0]),position=0, leave=True):
  
  if i < 8:
    val_set_x = np.vstack((np.reshape(partitioned_train[0:i],(-1, partitioned_train[0:i].shape[2])), np.reshape(partitioned_train[i+1:8], (-1, partitioned_train[i+1:8].shape[2]))))
    val_set_y = np.vstack((np.reshape(partitioned_train_target[0:i],(-1, partitioned_train_target[0:i].shape[1])), np.reshape(partitioned_train_target[i+1:8], (-1, partitioned_train_target[i+1:8].shape[1])))).flatten()
  else:
    val_set_x = np.vstack((np.reshape(partitioned_train[8:i],(-1, partitioned_train[8:i].shape[2])), np.reshape(partitioned_train[i+1:], (-1, partitioned_train[i+1:].shape[2]))))
    val_set_y = np.vstack((np.reshape(partitioned_train_target[8:i],(-1, partitioned_train_target[8:i].shape[1])), np.reshape(partitioned_train_target[i+1:], (-1, partitioned_train_target[i+1:].shape[1])))).flatten()
  
  maxScore = -np.Inf
  mean, stdev = stats.norm.fit(partitioned_train[i])
  part = (partitioned_train[i] - mean) / stdev
  val_set_x = (val_set_x - mean) / stdev

  for c in [0.01, 0.05, 0.1, 0.5, 1]:
    currentSVM = LinearSVC(C=c)
    currentSVM.fit(part, partitioned_train_target[i])
    
    cScore = currentSVM.score(val_set_x, val_set_y)
    if cScore > maxScore:
      maxScore = cScore
      bestEstimator = currentSVM
  
  tqdm.write(f'\nC:{bestEstimator.C}\tScore: {str(maxScore)}')
  svmArray.append(bestEstimator)

  6%|▌         | 1/17 [00:04<01:17,  4.83s/it]


C:0.01	Score: 0.7971428571428572


 12%|█▏        | 2/17 [00:09<01:11,  4.74s/it]


C:0.01	Score: 0.7747619047619048


 18%|█▊        | 3/17 [00:14<01:09,  4.98s/it]

...


C:0.01	Score: 0.7670833333333333


 94%|█████████▍| 16/17 [01:15<00:04,  4.62s/it]


C:0.01	Score: 0.7688888888888888


100%|██████████| 17/17 [01:20<00:00,  4.74s/it]


C:0.01	Score: 0.7872222222222223


### Global Classification Scheme

In [11]:
mean, stdev = stats.norm.fit(x_test)
x_test = (x_test - mean) / stdev

In [12]:
scores = []
char_epoch = 0

for c in tqdm(range(stimulus_code_test.shape[0]), position=0, leave=True):
  score = {}
  for i in range(12):
    currentScore = 0
    
    for b in range(15):
      index = c * 180 + i + 12 * b
      for k in range(len(svmArray)):
        currentScore += svmArray[k].predict(x_test[index:index+1])
    
    if not stimulus_code_test[c, i] in score:
      score[stimulus_code_test[c, i]] = currentScore
    else:
      score[stimulus_code_test[c, i]] += currentScore

  for k in score.keys():
    score[k] *= (1/15)*(1/17)

  scores.append(score)

100%|██████████| 100/100 [00:18<00:00,  5.44it/s]


In [ ]:
preds = []

for d in scores:
  maxRow = -np.Inf
  maxCol = 0
  rowIndex = colIndex = 0
  
  for k in d.keys():
    if k < 7 and d[k] > maxCol:
      maxCol = d[k]
      colIndex = k
    elif k >= 7 and d[k] > maxRow:
      maxRow = d[k]
      rowIndex = k
  
  preds.append(query_char(rowIndex, colIndex))

acc = accuracy_score(test_data_a['target'], preds)
print(acc)

### Single SVM -> Normalized Data

In [17]:
mean, stdev = stats.norm.fit(x_train)
x_train_new = (x_train - mean) / stdev

mean, stdev = stats.norm.fit(x_test)
x_test_new = (x_test - mean) / stdev

c = LinearSVC(C=0.01)
c.fit(x_train_new, y_train)
p = c.predict(x_test_new)

In [18]:
p_by_window = []

for c in range(100):
  row_cols = {}
  
  for w in range(180):
    if not stimulus_code_test[c,w] in row_cols:
      row_cols[stimulus_code_test[c,w]] = p[c * 180 + w]
    else:
      row_cols[stimulus_code_test[c,w]] += p[c * 180 + w]
  
  maxRow = -np.Inf
  maxCol = 0
  rowIndex = colIndex = 0
  
  for k in row_cols.keys():
    if k < 7 and row_cols[k] > maxCol:
      maxCol = row_cols[k]
      colIndex = k
    elif k >= 7 and row_cols[k] > maxRow:
      maxRow = row_cols[k]
      rowIndex = k
  
  p_by_window.append(query_char(rowIndex, colIndex))

acc_new = accuracy_score(test_data_a['target'], p_by_window)
print(acc_new)

0.3


### Bagging
__WARNING__: The execution of the cells below may take too much time, be cautious!

In [19]:
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(LinearSVC(C=0.01), n_estimators=17)
bag.fit(x_train_new, y_train)
pr = bag.predict(x_test)

In [20]:
pr = bag.predict(x_test_new)

In [21]:
pr_by_window = []

for c in range(100):
  row_cols = {}
  
  for w in range(180):
    if not stimulus_code_test[c,w] in row_cols:
      row_cols[stimulus_code_test[c,w]] = pr[c * 180 + w]
    else:
      row_cols[stimulus_code_test[c,w]] += pr[c * 180 + w]
  
  maxRow = -np.Inf
  maxCol = 0
  rowIndex = colIndex = 0
  
  for k in row_cols.keys():
    if k < 7 and row_cols[k] > maxCol:
      maxCol = row_cols[k]
      colIndex = k
    elif k >= 7 and row_cols[k] > maxRow:
      maxRow = row_cols[k]
      rowIndex = k
  
  pr_by_window.append(query_char(rowIndex, colIndex))

acc_new_final_lol = accuracy_score(test_data_a['target'], pr_by_window)
print(acc_new_final_lol)

0.33
